In [1]:
from lammps.calculator.worker import LammpsWorker
import asyncio
import time
loop = asyncio.get_event_loop()

In [2]:
workers = LammpsWorker(cwd="lammps_workers", num_workers=2)
loop.run_until_complete(workers.create())

In [4]:
script = """
            log  lammps.log
units  metal
dimension  3
boundary  p p p
atom_style  full
read_data  initial.data
kspace_style  pppm 0.000010
pair_style  buck/coul/long 10
pair_coeff 1 1 1309362.2766468062 0.104 0.0
pair_coeff 1 2 9892.357 0.20199 0.0
pair_coeff 2 2 2145.7345 0.3 30.2222
set type 2 charge -1.400000
set type 1 charge 1.400000
dump  1 all custom 1 mol.lammpstrj id type x y z fx fy fz
dump_modify  1 sort id
thermo_style  custom step etotal pxx pyy pzz pxy pxz pyz
run  0
"""

files = {
    'initial.data': b"""pymatgen autogenerated data file

8 atoms

2 atom types

0 4.1990858 xlo xhi
0 4.1990858 ylo yhi
0 4.1990858 zlo zhi
2.571198492157551e-16 2.571198492157551e-16 2.5711984921575517e-16 xy xz yz

Masses

1 24.305
2 15.9994

Atoms

1 1 1 1.4 0.0 0.0 0.0
2 1 1 1.4 3.3763224914577695e-16 2.0995429 2.0995429
3 1 1 1.4 2.0995429 0.0 2.0995429
4 1 1 1.4 2.0995429000000003 2.0995429 2.571198492157551e-16
5 1 2 -1.4 2.0995429000000003 2.0995429 2.0995429000000003
6 1 2 -1.4 2.0995429 0.0 1.2855992460787756e-16
7 1 2 -1.4 3.3763224914577695e-16 2.0995429 1.2855992460787756e-16
8 1 2 -1.4 0.0 0.0 2.0995429
"""
}

async def run_lammps_job():
    jobs = []
    
    start_time = time.perf_counter()
    for _ in range(100): 
        jobs.append(await workers.submit(script, files, properties={'stress', 'forces', 'energy'}))
    results = await asyncio.gather(*[_.future for _ in jobs])
    print('time', 100, time.perf_counter() - start_time)

job = loop.run_until_complete(run_lammps_job())

time 100 0.49794309999924735


In [4]:
job.results

{'energy': -85.34101,
 'forces': [[1.77636e-15, 1.40513e-15, 1.9984e-15],
  [8.88178e-16, 1.08802e-14, 1.72085e-15],
  [1.63758e-14, -8.70831e-15, 1.26808e-15],
  [9.67976e-16, -2.35922e-16, 4.4912e-15],
  [-6.04378e-15, 1.20806e-14, -1.62717e-14],
  [-3.27516e-15, -2.29677e-15, -4.42354e-16],
  [-1.33227e-15, -2.44249e-15, 8.88178e-16],
  [-8.88178e-15, -1.113e-14, -9.99201e-16]],
 'stress': [[661.01685, 9.2077572e-11, -3.5735631e-10],
  [9.2077572e-11, 661.01685, -4.0058502e-10],
  [-3.5735631e-10, -4.0058502e-10, 661.01685]]}